In [1]:
# Add project root to sys.path so 'src' is importable
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
print("Project root in sys.path:", project_root)
print("Current working directory:", os.getcwd())

Project root in sys.path: c:\Users\wbrya\OneDrive\Documents\GitHub\AI-CFO-FYP
Current working directory: c:\Users\wbrya\OneDrive\Documents\GitHub\AI-CFO-FYP\notebooks


In [2]:
# Cell 2: Load environment variables
from dotenv import load_dotenv
import os

load_dotenv()

# Verify the key is loaded (prints part of the key)
openai_key = os.environ.get("OPENAI_API_KEY")
if openai_key:
    print(f"OpenAI API Key loaded: {openai_key[:3]}...{openai_key[-4:]}")
else:
    print("OpenAI API Key not loaded. Please check your .env file.")
    # You might want to raise an error or stop here if the key is essential

OpenAI API Key loaded: sk-...z-8A


In [3]:

from src.llm.OpenAIClient import OpenAIClient

try:
    # Instantiate the client
    openai_client = OpenAIClient()

    # Prepare some text to embed
    texts_to_embed = [
        "This is a sentence about financial performance.",
        "Balance sheet assets and liabilities.",
    ]

    print("\nAttempting to get embeddings...")

    # Get embeddings
    embeddings = openai_client.get_embeddings(texts_to_embed)

    # Print results
    print(f"Successfully generated {len(embeddings)} embeddings.")

    # Check the first embedding
    if embeddings:
        print(f"Length of the first embedding: {len(embeddings[0])}")
        print(f"First 10 dimensions of the first embedding: {embeddings[0][:10]}...")
        print(f"Length of the second embedding: {len(embeddings[1])}")
        print(f"First 10 dimensions of the second embedding: {embeddings[1][:10]}...")
    else:
        print("No embeddings were returned.")

except ValueError as e:
    print(f"Configuration Error: {e}")
except Exception as e:
    print(f"An error occurred during embedding: {e}")

Initialized OpenAI client with model: text-embedding-3-small

Attempting to get embeddings...
Successfully generated 2 embeddings.
Length of the first embedding: 1536
First 10 dimensions of the first embedding: [0.022943295538425446, 0.02701064571738243, 0.03878304734826088, 0.03408554568886757, 0.0431368313729763, 0.02771240659058094, -0.028800852596759796, 0.019076447933912277, 0.01999303326010704, -0.003376330714672804]...
Length of the second embedding: 1536
First 10 dimensions of the second embedding: [0.024601895362138748, 0.021060355007648468, 0.05293421074748039, 0.009420495480298996, 0.015228619799017906, 0.023303329944610596, 0.0007732361555099487, 0.0009776124497875571, 0.01635010726749897, -0.021131185814738274]...
